In [1]:
# install pyspark
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 28.2 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u312-b07-0ubuntu1~18.04 [28.2 MB]
Fetched 28.2 MB in 1s (19.3 MB/s)
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 156210 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Setting up openjdk-8-jre-headless:amd64 (8u312-b07-0ubunt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


**Q1**

> Find the maximal delays (you should consider both ArrDelay and DepDelay) for each month of 2007.

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [4]:
spark

In [35]:
df = spark.read.csv("/content/gdrive/MyDrive/bigdata_hw2/2007.csv", header=True, inferSchema=True)

In [26]:
df.printSchema()
df.show(5)

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- C

In [5]:
from pyspark.sql.functions import *

In [36]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for i in range(1, 13):
  print(months[i - 1] + ': ')
  print('DepDelay')
  df_month = df.filter(df.Month == i)
  df_month_DepDelay = df_month.filter("DepDelay != 'NA'")
  df_month_DepDelay = df_month_DepDelay.withColumn("DepDelay", df_month_DepDelay["DepDelay"].cast('int'))
  df_month_DepDelay.select(["DepDelay"]).orderBy(desc("DepDelay")).show(5)

  print('ArrDelay')
  df_month_ArrDelay = df_month.filter("ArrDelay != 'NA'")
  df_month_ArrDelay = df_month_ArrDelay.withColumn("ArrDelay", df_month_ArrDelay["ArrDelay"].cast('int'))
  df_month_ArrDelay.select(["ArrDelay"]).orderBy(desc("ArrDelay")).show(5)

Jan: 
DepDelay
+--------+
|DepDelay|
+--------+
|    1406|
|    1255|
|    1199|
|    1198|
|    1194|
+--------+
only showing top 5 rows

ArrDelay
+--------+
|ArrDelay|
+--------+
|    1426|
|    1209|
|    1202|
|    1201|
|    1200|
+--------+
only showing top 5 rows

Feb: 
DepDelay
+--------+
|DepDelay|
+--------+
|    1340|
|    1328|
|    1307|
|    1267|
|    1237|
+--------+
only showing top 5 rows

ArrDelay
+--------+
|ArrDelay|
+--------+
|    1359|
|    1330|
|    1302|
|    1261|
|    1231|
+--------+
only showing top 5 rows

Mar: 
DepDelay
+--------+
|DepDelay|
+--------+
|    1547|
|    1340|
|    1333|
|    1290|
|    1190|
+--------+
only showing top 5 rows

ArrDelay
+--------+
|ArrDelay|
+--------+
|    1564|
|    1349|
|    1348|
|    1331|
|    1213|
+--------+
only showing top 5 rows

Apr: 
DepDelay
+--------+
|DepDelay|
+--------+
|    1415|
|    1372|
|    1142|
|    1135|
|    1125|
+--------+
only showing top 5 rows

ArrDelay
+--------+
|ArrDelay|
+--------+
|  

**Q2**

> How many flights were delayed caused by security between 2000 ~ 2005? Please
show the counting for each year.



In [29]:
files = ['2000.csv', '2001.csv', '2002.csv', '2003.csv', '2004.csv', '2005.csv']
total_SecurityDelay_count = 0
for file in files:
  fp = "/content/gdrive/MyDrive/bigdata_hw2/" + file
  df_SecurityDelay = spark.read.csv(fp, header=True, inferSchema=True)
  df_SecurityDelay.na.fill(0)
  df_SecurityDelay = df_SecurityDelay.withColumn("SecurityDelay", df_SecurityDelay["SecurityDelay"].cast('int'))
  df_SecurityDelay = df_SecurityDelay.filter("SecurityDelay > 0")

  tmp = df_SecurityDelay.count()
  total_SecurityDelay_count += tmp
  print(file[0:4] + ' has ' + str(tmp) + ' SecurityDelay.')
  if tmp != 0:
    df_SecurityDelay.select(["Year", "Month", "SecurityDelay"]).orderBy(desc("SecurityDelay")).show(5)

print("there are " + str(total_SecurityDelay_count) + " SecurityDelays between 2000 - 2005.")

2000 has 0 SecurityDelay.
2001 has 0 SecurityDelay.
2002 has 0 SecurityDelay.
2003 has 3740 SecurityDelay.
+----+-----+-------------+
|Year|Month|SecurityDelay|
+----+-----+-------------+
|2003|   12|          230|
|2003|    6|          218|
|2003|    6|          214|
|2003|   11|          208|
|2003|    6|          204|
+----+-----+-------------+
only showing top 5 rows

2004 has 8158 SecurityDelay.
+----+-----+-------------+
|Year|Month|SecurityDelay|
+----+-----+-------------+
|2004|    8|          533|
|2004|    8|          451|
|2004|    8|          382|
|2004|    8|          380|
|2004|    9|          312|
+----+-----+-------------+
only showing top 5 rows

2005 has 6627 SecurityDelay.
+----+-----+-------------+
|Year|Month|SecurityDelay|
+----+-----+-------------+
|2005|    8|          326|
|2005|    7|          294|
|2005|    8|          282|
|2005|   12|          261|
|2005|    7|          259|
+----+-----+-------------+
only showing top 5 rows

there are 18525 SecurityDelays 

**Q3**
> List Top 5 airports which occur delays most and least in 2008. (Please show the IATA airport code)

In [6]:
fp = "/content/gdrive/MyDrive/bigdata_hw2/2008.csv"
df = spark.read.csv(fp, header=True, inferSchema=True)

df_DepDelay = df.select(["Origin", "DepDelay"])
df_ArrDelay = df.select(["Dest", "ArrDelay"])

# clear na
df_DepDelay.na.fill(0)
df_ArrDelay.na.fill(0)

df_DepDelay = df_DepDelay.filter("DepDelay > 0")
df_ArrDelay = df_ArrDelay.filter("ArrDelay > 0")

origin_delays = df_DepDelay.withColumn("IATA", df_DepDelay["Origin"])
print("most 5 origin delays")
top_origin = origin_delays.groupBy("IATA").count().orderBy(desc("count")).show(5)
print("least 5 origin delays")
least_origin = origin_delays.groupBy("IATA").count().orderBy("count").show(5)

dest_delays = df_ArrDelay.withColumn("IATA", df_ArrDelay["Dest"])
print("most 5 dest delays")
top_dest = dest_delays.groupBy("IATA").count().orderBy(desc("count")).show(5)
print("least 5 dest delays")
least_dest = dest_delays.groupBy("IATA").count().orderBy("count").show(5)

total_delays = origin_delays.union(dest_delays)
print("most 5 total delays")
top_total = total_delays.groupBy("IATA").count().orderBy(desc("count")).show(5)
print("least 5 total delays")
least_total = total_delays.groupBy("IATA").count().orderBy("count").show(5)

most 5 origin delays
+----+------+
|IATA| count|
+----+------+
| ATL|175017|
| ORD|159427|
| DFW|127749|
| DEN|104414|
| LAX| 87258|
+----+------+
only showing top 5 rows

least 5 origin delays
+----+-----+
|IATA|count|
+----+-----+
| TUP|    1|
| INL|    1|
| PUB|    2|
| PIR|    3|
| BJI|    4|
+----+-----+
only showing top 5 rows

most 5 dest delays
+----+------+
|IATA| count|
+----+------+
| ATL|187243|
| ORD|151871|
| DFW|119817|
| DEN|102681|
| LAX| 93508|
+----+------+
only showing top 5 rows

least 5 dest delays
+----+-----+
|IATA|count|
+----+-----+
| TUP|    2|
| PIR|    4|
| BJI|   12|
| INL|   17|
| SUX|   35|
+----+-----+
only showing top 5 rows

most 5 total delays
+----+------+
|IATA| count|
+----+------+
| ATL|362260|
| ORD|311298|
| DFW|247566|
| DEN|207095|
| LAX|180766|
+----+------+
only showing top 5 rows

least 5 total delays
+----+-----+
|IATA|count|
+----+-----+
| PUB|    2|
| TUP|    3|
| PIR|    7|
| BJI|   16|
| INL|   18|
+----+-----+
only showing top 5 rows